In [1]:
import numerapi
NAPI = numerapi.NumerAPI(verbosity="info")
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import mean_absolute_error
import os

DIR = "data"
def download_current_data(directory: str):
        """
        Downloads the data for the current round
        :param directory: The path to the directory where the data needs to be saved
        """
        current_round = NAPI.get_current_round()
        if os.path.isdir(f'{directory}/numerai_dataset_{current_round}/'):
            print(f"You already have the newest data! Current round is: {current_round}")
        else:
            print(f"Downloading new data for round: {current_round}!")
            NAPI.download_current_dataset(dest_path=directory, unzip=True)

def load_data(directory: str, reduce_memory: bool=True) -> tuple:
        """
        Get data for current round
        :param directory: The path to the directory where the data needs to be saved
        :return: A tuple containing the datasets
        """
        print('Loading the data')
        full_path = f'{directory}/numerai_dataset_{NAPI.get_current_round()}/'
        train_path = full_path + 'numerai_training_data.csv'
        test_path = full_path + 'numerai_tournament_data.csv'
        train = pd.read_csv(train_path)
        test = pd.read_csv(test_path)
        # Reduce all features to 32-bit floats
        if reduce_memory:
            num_features = [f for f in train.columns if f.startswith("feature")]
            train[num_features] = train[num_features].astype(np.float32)
            test[num_features] = test[num_features].astype(np.float32)
        val = test[test['data_type'] == 'validation']
        test = test[test['data_type'] != 'validation']
        return train, val, test
    # Download, unzip and load data
download_current_data(DIR)
train, val, test = load_data(DIR, reduce_memory=True)

You already have the newest data! Current round is: 253
Loading the data


In [2]:
features_list = [c for c in train.columns if str(c).startswith("feature")]

In [3]:
def get_group_stats(df: pd.DataFrame) -> pd.DataFrame:
        for group in ["intelligence", "wisdom", "charisma", "dexterity", "strength", "constitution"]:
            cols = [col for col in df.columns if group in col]
            df[f"feature_{group}_mean"] = df[cols].mean(axis=1)
            df[f"feature_{group}_std"] = df[cols].std(axis=1)
            df[f"feature_{group}_skew"] = df[cols].skew(axis=1)
        return df
train = get_group_stats(train)
val = get_group_stats(val)
test = get_group_stats(test)

In [4]:
from sklearn import preprocessing
ft_corr_list = ['feature_dexterity7', 'feature_charisma18', 'feature_charisma63', 'feature_dexterity14']#Please try other features!
interactions = preprocessing.PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

# These can be condensed into fit_transform
interactions.fit(train[ft_corr_list], train["target"])
X_train_interact = pd.DataFrame(interactions.transform(train[ft_corr_list]))
X_best_val_inter = pd.DataFrame(interactions.transform(val[ft_corr_list]))
X_best_test_inter = pd.DataFrame(interactions.transform(test[ft_corr_list]))

train = pd.concat([train,X_train_interact],axis=1)

val = val.reset_index().drop(columns='index')
val = pd.concat([val,X_best_val_inter],axis=1)

test = test.reset_index().drop(columns='index')
test = pd.concat([test,X_best_test_inter],axis=1)

In [5]:
train.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,0,1,2,3,4,5,6,7,8,9
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,...,1.00,0.50,0.50,0.50,0.5000,0.5000,0.5000,0.2500,0.2500,0.250
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,...,0.25,0.75,0.25,0.00,0.1875,0.0625,0.0000,0.1875,0.0000,0.000
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,...,1.00,0.25,0.50,0.50,0.2500,0.5000,0.5000,0.1250,0.1250,0.250
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,...,0.25,0.75,1.00,0.25,0.1875,0.2500,0.0625,0.7500,0.1875,0.250
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,...,0.75,0.50,0.75,0.50,0.3750,0.5625,0.3750,0.3750,0.2500,0.375


In [7]:
val.drop("prediction", axis=1, inplace=True)

In [8]:
dtrain = lgb.Dataset(train[train.columns.drop('id').drop('era').drop('data_type').drop('target')].fillna(0), label=train["target"])
dvalid = lgb.Dataset(val[val.columns.drop('id').drop('era').drop('data_type').drop('target')].fillna(0), label=val["target"])

best_config ={"objective":"regression", "num_leaves":31,"learning_rate":0.01,"n_estimators":2000,"max_depth":5,"metric":"mse","verbosity": 10, "random_state": 0} 

model = lgb.train(best_config, dtrain)

train.loc[:, "prediction"] = model.predict(train[train.columns.drop('id').drop('era').drop('data_type').drop('target')])

val.loc[:,"prediction"] = val["target"]
val.loc[:,"prediction"] = model.predict(val[train.columns.drop('id').drop('era').drop('data_type').drop('target')])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.792150
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.184534
[LightGBM] [Debug] init for col-wise cost 0.068321 seconds, init for row-wise cost 0.849755 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.218899 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 5682
[LightGBM] [Info] Number of data points in the train set: 501808, number of used features: 339
[LightGBM] [Info] Start training from score 0.499997
[LightGBM] [Debug] Trained 

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree 

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree 

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree 

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree 

...

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree 

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree 

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 29 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 30 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Tra

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 30 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a

[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 29 and max_depth = 5
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 5
[LightGBM] [Debug] Trained a

In [36]:
def sharpe_ratio(corrs: pd.Series) -> np.float32:
        """
        Calculate the Sharpe ratio for Numerai by using grouped per-era data

        :param corrs: A Pandas Series containing the Spearman correlations for each era
        :return: A float denoting the Sharpe ratio of your predictions.
        """
        return corrs.mean() / corrs.std()


def evaluate(df: pd.DataFrame) -> tuple:
        """
        Evaluate and display relevant metrics for Numerai 

        :param df: A Pandas DataFrame containing the columns "era", "target" and a column for predictions
        :param pred_col: The column where the predictions are stored
        :return: A tuple of float containing the metrics
        """
        def _score(sub_df: pd.DataFrame) -> np.float32:
            """Calculates Spearman correlation"""
            return spearmanr(sub_df["target"], sub_df["prediction"])[0]

        # Calculate metrics
        corrs = df.groupby("era").apply(_score)
        print(corrs)
        payout_raw = (corrs / 0.2).clip(-1, 1)
        spearman = round(corrs.mean(), 4)

        payout = round(payout_raw.mean(), 4)
        numerai_sharpe = round(sharpe_ratio(corrs), 4)
        mae = mean_absolute_error(df["target"], df["prediction"]).round(4)

        # Display metrics
        print(f"Spearman Correlation: {spearman}")
        print(f"Average Payout: {payout}")
        print(f"Sharpe Ratio: {numerai_sharpe}")
        print(f"Mean Absolute Error (MAE): {mae}")
        return spearman, payout, numerai_sharpe, mae
        
feature_spearman_val = [spearmanr(val["prediction"], val[f])[0] for f in features_list]
feature_exposure_val = np.std(feature_spearman_val).round(4)
spearman, payout, numerai_sharpe, mae = evaluate(val)

era
era121    0.911763
era122    0.912419
era123    0.912402
era124    0.912730
era125    0.912741
era126    0.912891
era127    0.913002
era128    0.912858
era129    0.912650
era130    0.912933
era131    0.912610
era132    0.912627
era197    0.914198
era198    0.913355
era199    0.913769
era200    0.914101
era201    0.913851
era202    0.913442
era203    0.913583
era204    0.914069
era205    0.912922
era206    0.912315
era207    0.913835
era208    0.914425
era209    0.913255
era210    0.911896
era211    0.913319
era212    0.912439
dtype: float64
Spearman Correlation: 0.9131
Average Payout: 1.0
Sharpe Ratio: 1301.8701
Mean Absolute Error (MAE): 0.06


In [44]:
test.loc[:, "prediction"] = 0

test.loc[:, "prediction"] = model.predict(test.drop(['era', 'data_type', 'prediction', 'id'], axis=1))
test[['id', "prediction"]].to_csv("submission_test.csv", index=False)

val[['id', "prediction"]].to_csv("submission_val.csv", index=False)

test=0
val=0

directory = "data"
full_path = f'{directory}/numerai_dataset_{NAPI.get_current_round()}/'

test_path = full_path + 'numerai_tournament_data.csv'

tournament_data = pd.read_csv(test_path)
tournament_data_id=tournament_data['id']
tournament_data_id2=tournament_data['feature_dexterity7']
tournament_data_id=pd.concat([tournament_data_id,tournament_data_id2],axis=1)

val=pd.read_csv("submission_val.csv")
test=pd.read_csv("submission_test.csv")

test_val_concat=pd.concat([val[['id', "prediction"]],test[['id', "prediction"]]],axis=0).set_index('id')
tournament_data_id=tournament_data_id.set_index('id')

conc_submit=pd.concat([tournament_data_id,test_val_concat],axis=1).drop(columns='feature_dexterity7').reset_index()
conc_submit=conc_submit.rename(columns={'index': 'id'})
conc_submit.to_csv("submission_file"+".csv", index=False)

In [46]:
def neutralize(series,by, proportion):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)
    exposures = np.hstack((exposures, np.array([np.mean(series)] * len(exposures)).reshape(-1, 1)))
    correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores)[0]))
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized
    
by=pd.read_csv('data/numerai_dataset_'+str(NAPI.get_current_round())+'/example_predictions.csv')

neut=pd.read_csv("submission_file.csv")
neut=pd.DataFrame({'prediction':neutralize(neut['prediction'],by['prediction'], 0.3)})

conc=pd.concat([by.drop(columns="prediction"),neut],axis=1)
conc.to_csv("neutralized_submission_file.csv", index=False)#submission file

<ipython-input-46-db7083cc578f>:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  correction = proportion * (exposures.dot(np.linalg.lstsq(exposures, scores)[0]))


In [52]:
conc.to_csv("neutralized_submission_file.csv")#submission file

In [9]:
conc

NameError: name 'conc' is not defined